In [29]:
import gurobipy as gp
from gurobipy import GRB
import json
from pathlib import Path
import pandas as pd

In [30]:
dimension = "5_30_instances"
folder_projects = Path("instances_projects")
filename_projects = "generic_5_30_projects_0.csv"
filepath_projects = folder_projects / dimension / filename_projects
folder_students = Path("instances_students")
filename_students = "generic_5_30_students_0.csv"
filepath_students = folder_students / dimension / filename_students
projects_info = pd.read_csv(filepath_projects)
students_info = pd.read_csv(filepath_students)
students_info["fav_partners"] = students_info["fav_partners"].apply(lambda x: set(json.loads(x)))
students_info["project_prefs"] = students_info["project_prefs"].apply(lambda x: tuple(json.loads(x)))

In [31]:
print(projects_info)

                        name  desired#groups  max#groups  ideal_group_size  \
0                 Accounting               4           5                 3   
1      Production Management               3           4                 4   
2         Financial Planning               3           6                 4   
3        Business Statistics               3           5                 2   
4  Human Resource Management               4           6                 4   

   min_group_size  max_group_size  pen_groups  pen_size  
0               2               5           1         3  
1               1               6           1         2  
2               2               6           3         2  
3               1               4           2         1  
4               1               5           1         3  


In [32]:
print(students_info)

                  name  fav_partners    project_prefs
0     Kathleen Sanders  {18, 13, 23}  (3, 1, 3, 3, 0)
1           Susan Diaz   {19, 3, 23}  (0, 0, 2, 1, 2)
2           Kelly Ross   {1, 11, 23}  (0, 0, 1, 1, 2)
3         Pamela Jones   {1, 26, 10}  (0, 0, 2, 1, 2)
4      Brandon Mendoza    {9, 5, 25}  (2, 3, 0, 2, 2)
5       John Rodriguez    {4, 6, 14}  (2, 3, 0, 2, 2)
6        Logan Sanders   {29, 28, 5}  (2, 3, 0, 1, 2)
7         Willie White    {0, 4, 23}  (2, 1, 2, 2, 1)
8           Juan Kelly  {27, 11, 14}  (3, 1, 0, 0, 1)
9          Larry Smith   {24, 19, 6}  (2, 2, 1, 1, 2)
10  Christopher Morris    {3, 6, 14}  (1, 1, 1, 1, 1)
11       Andrew Cooper   {2, 20, 22}  (1, 1, 1, 1, 1)
12     Jeffrey Alvarez    {1, 4, 13}  (1, 2, 1, 2, 2)
13        Carl Alvarez  {25, 26, 12}  (1, 2, 1, 2, 2)
14       Amanda Torres  {10, 19, 21}  (2, 2, 1, 1, 1)
15        Hannah Gomez   {4, 14, 23}  (2, 2, 0, 2, 2)
16        Gregory Reed   {10, 27, 6}  (1, 1, 0, 1, 2)
17       Ashley Bailey   {1,

In [33]:
reward_bilateral = 2
penalty_unassignment = 3

In [34]:

model = gp.Model()

Set parameter Username
Set parameter LicenseID to value 2654427
Academic license - for non-commercial use only - expires 2026-04-19


In [35]:
project_ids = range(len(projects_info))
student_ids = range(len(students_info))

projects_group_ids = {
    project_id: range(projects_info["max#groups"][project_id])
    for project_id in project_ids
}


In [36]:
x = model.addVars(
    (
        (project_id, group_id, student_id)
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
        for student_id in student_ids
    ),
    vtype=GRB.BINARY,
    name="assign",
)
model.update()

In [37]:
y = model.addVars(
    (
        (project_id, group_id)
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ),
    vtype=GRB.BINARY,
    name="establish_group",
)

In [38]:
favorite_partners_students = students_info["fav_partners"].tolist()
mutual_pairs = {
    (student_id, partner_id)
    for student_id, favorite_partners in enumerate(favorite_partners_students)
    for partner_id in favorite_partners
    if student_id < partner_id and
    student_id in favorite_partners_students[partner_id]
}

In [39]:
z = model.addVars(mutual_pairs, vtype=GRB.BINARY, name="not_realize_bilateral_cooperation_wish")

In [40]:
v = model.addVars(student_ids, name = "student_unassigned")

In [41]:
gs_surplus = model.addVars(
    (
        (project_id, group_id)
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ),
    name="group_size_surplus",
)

In [42]:
gs_deficit = model.addVars(
    (
        (project_id, group_id)
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ),
    name="group_size_deficit",
)

In [43]:
project_preferences = {
    (student_id, project_id): students_info["project_prefs"][student_id][project_id]
    for student_id in student_ids
    for project_id in project_ids
}

In [44]:
rewards_student_project_preference = gp.quicksum(
    project_preferences[student_id, project_id] * x[project_id, group_id, student_id]
    for project_id in project_ids
    for group_id in projects_group_ids[project_id]
    for student_id in student_ids
)

rewards_bilateral_cooperation_wish_realized = reward_bilateral * gp.quicksum(
    1 - z[*mutual_pair] for mutual_pair in mutual_pairs
)

penalties_student_unassigned = penalty_unassignment * gp.quicksum(v.values())

penalties_more_groups_than_offered = gp.quicksum(
    projects_info["pen_groups"][project_id] * y[project_id, group_id]
    for project_id in project_ids
    for group_id in projects_group_ids[project_id]
    if group_id >= projects_info["desired#groups"][project_id]
)

penalties_not_ideal_group_size = gp.quicksum(
    projects_info["pen_size"][project_id]
    * (gs_surplus[project_id, group_id] + gs_deficit[project_id, group_id])
    for project_id in project_ids
    for group_id in projects_group_ids[project_id]
)

model.setObjective(
    rewards_student_project_preference
    + rewards_bilateral_cooperation_wish_realized
    - penalties_student_unassigned
    - penalties_more_groups_than_offered
    - penalties_not_ideal_group_size,
    sense=GRB.MAXIMIZE,
)

In [45]:
model.addConstrs(
    (x.sum("*", "*", student_id) + v[student_id] == 1 for student_id in student_ids),
    name="penalty_if_unassigned",
)
model.update()

In [46]:
model.addConstrs(
    (
        y[project_id, group_id] <= y[project_id, group_id - 1]
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
        if group_id > 0
    ), name="only_consecutive_group_ids"
)
model.update()

In [47]:
model.addConstrs(
    (
        x.sum(project_id, group_id, "*")
        >= projects_info["min_group_size"][project_id] * y[project_id, group_id]
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ), name="ensure_min_group_size"
)
model.update()

In [48]:
model.addConstrs(
    (
        x.sum(project_id, group_id, "*")
        <= projects_info["max_group_size"][project_id] * y[project_id, group_id]
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ), name="cap_group_size_at_max"
)
model.update()

In [49]:
model.addConstrs(
    (
        gs_surplus[project_id, group_id]
        >= x.sum(project_id, group_id, "*")
        - projects_info["ideal_group_size"][project_id]
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ), name="ensure_correct_group_size_surplus"
)
model.update()

In [50]:
model.addConstrs(
    (
        gs_deficit[project_id, group_id]
        >= projects_info["ideal_group_size"][project_id]
        - x.sum(project_id, group_id, "*")
        - projects_info["max_group_size"][project_id] * (1 - y[project_id, group_id])
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ), name="ensure_correct_group_size_deficit"
)
model.update()

In [51]:
max_num_groups = max(projects_info["max#groups"])


unique_group_identifiers = {
    (project_id, group_id): project_id + group_id / max_num_groups
    for project_id in project_ids
    for group_id in projects_group_ids[project_id]
}

In [52]:
num_projects = len(projects_info)

model.addConstrs(
    (
        z[first_student_id, second_student_id] * num_projects
        >= sum(
            unique_group_identifiers[project_id, group_id]
            * (
                x[project_id, group_id, first_student_id]
                - x[project_id, group_id, second_student_id]
            )
            for project_id in project_ids
            for group_id in projects_group_ids[project_id]
        )
        for (first_student_id, second_student_id) in mutual_pairs
    ), name="ensure_correct_inidicator_different_group_1"
)
model.update()

In [53]:
model.addConstrs(
    (
        z[first_student_id, second_student_id] * num_projects
        >= sum(
            unique_group_identifiers[project_id, group_id]
            * (
                x[project_id, group_id, second_student_id]
                - x[project_id, group_id, first_student_id]
            )
            for project_id in project_ids
            for group_id in projects_group_ids[project_id]
        )
        for (first_student_id, second_student_id) in mutual_pairs
    ), name="ensure_correct_inidicator_different_group_2"
)
model.update()

In [54]:
model.Params.TimeLimit = 60
model.optimize()

Set parameter TimeLimit to value 60
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-12500H, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  60

Optimize a model with 203 rows, 912 columns and 6550 nonzeros
Model fingerprint: 0x2e395b9f
Variable types: 82 continuous, 830 integer (830 binary)
Coefficient statistics:
  Matrix range     [2e-01, 6e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective -42.0000000
Presolve removed 26 rows and 0 columns
Presolve time: 0.02s
Presolved: 177 rows, 912 columns, 5770 nonzeros
Variable types: 0 continuous, 912 integer (860 binary)

Root relaxation: objective 1.084312e+02, 390 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work

In [55]:
for project_id in project_ids:
    print("\n")
    print(projects_info["name"][project_id])
    for group_id in projects_group_ids[project_id]:
        print("\nGroup", group_id + 1)
        for student_id in student_ids:
            if x[project_id, group_id, student_id].X > 0.5:
                print(students_info["name"][student_id], student_id)



Accounting

Group 1
Juan Kelly 8
Gregory Reed 16
Laura Mitchell 27

Group 2
Kathleen Sanders 0
Emily Parker 18
Gerald Gutierrez 22

Group 3

Group 4

Group 5


Production Management

Group 1
Brandon Mendoza 4
John Rodriguez 5
Logan Sanders 6
Jose Roberts 29

Group 2

Group 3

Group 4


Financial Planning

Group 1
Amanda Torres 14
Ashley Bailey 17
Frances Adams 19
Roy King 21

Group 2

Group 3

Group 4

Group 5

Group 6


Business Statistics

Group 1

Group 2

Group 3

Group 4

Group 5


Human Resource Management

Group 1
Jeffrey Alvarez 12
Carl Alvarez 13
Aaron Myers 25
Bobby Smith 28

Group 2
Kelly Ross 2
Larry Smith 9
Andrew Cooper 11
Frank Ramos 24

Group 3
Pamela Jones 3
Christopher Morris 10
Tyler Carter 20
Lucas Thomas 26

Group 4
Susan Diaz 1
Willie White 7
Hannah Gomez 15
Stephen Hernandez 23

Group 5

Group 6


In [56]:
model.dispose()
gp.disposeDefaultEnv()

Freeing default Gurobi environment
